# 量化（Quantization）

量化是指将神经网络中的权重和激活从高精度（如float32）转换为低精度（如int8），以减少模型大小、加速推理并降低能耗。

## 1. 量化的基本原理与动机

- **减少模型存储和内存占用**
- **加速推理速度**
- **降低功耗，适合部署在移动端和嵌入式设备**

## 2. 量化的主要类型

- **权重量化**：只对权重进行量化
- **激活量化**：对中间激活值进行量化
- **后训练量化（PTQ）**：训练后直接量化
- **量化感知训练（QAT）**：训练时模拟量化误差

## 3. 权重量化、激活量化的底层实现

### (1) 量化过程

$$
\begin{aligned}
\text{scale} &= \frac{\text{max\_val} - \text{min\_val}}{2^{\text{num\_bits}} - 1}, \\
\text{zero\_point} &= \text{qmin} - \frac{\text{min\_val}}{\text{scale}}, \\
q_x &= \text{round}\left(\frac{x}{\text{scale}} + \text{zero\_point}\right), \\
q_{\text{final}} &= \text{clip}(q_x, \text{qmin}, \text{qmax}).
\end{aligned}
$$

### (2) 反量化过程
反量化通常在推理结束后需要将量化结果还原为浮点数（如输出展示、误差分析等场景）时用到。

$$
x_{\text{dequant}} = \text{scale} \times (q_{\text{final}} - \text{zero\_point}).
$$

In [1]:
import numpy as np

"""量化和反量化"""
def quantize_tensor(x, num_bits=8):
    qmin = 0 # 量化最小值和最大值，范围为[0, 2^num_bits - 1]
    qmax = 2 ** num_bits - 1
    min_val, max_val = x.min(), x.max() # 原始张量的最小/最大值
    # 计算量化和反量化参数
    scale = (max_val - min_val) / (qmax - qmin) # 缩放因子
    zero_point = qmin - min_val / scale # 零点偏移
    # 量化公式: q_x = round(x / scale + zero_point)
    q_x = zero_point + x / scale
    q_x = np.clip(np.round(q_x), qmin, qmax) # 限制在 [qmin, qmax] 范围内
    return q_x.astype(np.uint8), scale, zero_point # 转换为 uint8 类型

def dequantize_tensor(q_x, scale, zero_point):
    # 反量化公式: x_dequant = scale * (q_x - zero_point)
    return scale * (q_x.astype(np.float32) - zero_point)

# 示例：对一个张量进行8位量化和反量化
x = np.random.randn(10).astype(np.float32)
q_x, scale, zero_point = quantize_tensor(x)
x_dequant = dequantize_tensor(q_x, scale, zero_point)

print('原始张量:', x)
print('量化后:', q_x)
print('反量化后:', x_dequant)

原始张量: [ 0.46483758  1.1436542  -1.457284   -0.658385    1.2318981   0.6196174
  0.95771927  0.00453585 -0.02097602  1.118749  ]
量化后: [182 247   0  76 255 197 229 139 136 244]
反量化后: [ 0.4620538   1.1475316  -1.4572839  -0.6558022   1.2318981   0.620241
  0.957707    0.00858392 -0.02305352  1.1158942 ]


## 4. PyTorch中的后训练量化（PTQ）

PyTorch提供了简单的API来对模型进行后训练量化。

In [6]:
import torch
import torch.nn as nn
import torch.ao.quantization as quantization

torch.backends.quantized.engine = 'qnnpack'  # 适配 Apple Silicon

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(4, 2) # 定义一个4输入2输出的全连接层
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()
model.eval() # 设置为评估模式（不计算梯度）


# 设置量化配置
model.qconfig = quantization.get_default_qconfig('qnnpack')

# 插入量化/反量化层（准备模型
torch.quantization.prepare(model, inplace=True)
# 示例数据，这里通常需要用一批数据跑一遍模型以收集量化参数
example_input = torch.randn(10, 4)
# 前向传播收集参数
model(example_input)
# 转换为量化模型
torch.quantization.convert(model, inplace=True)
print(model)
print(model.fc.weight().dtype)

SimpleModel(
  (fc): QuantizedLinear(in_features=4, out_features=2, scale=0.008065156638622284, zero_point=130, qscheme=torch.per_tensor_affine)
)
torch.qint8


## 5. PyTorch中的量化感知训练（QAT）

QAT在训练过程中模拟量化误差，能获得更高精度的量化模型。

In [8]:
# 量化感知训练示例
qat_model = SimpleModel()
qat_model.qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')
torch.quantization.prepare_qat(qat_model, inplace=True)

# 模拟训练过程
optimizer = torch.optim.SGD(qat_model.parameters(), lr=0.01)
for _ in range(5):
    optimizer.zero_grad()
    output = qat_model(torch.randn(10, 4))
    loss = output.sum()
    loss.backward()
    optimizer.step()

# 转换为量化模型
torch.quantization.convert(qat_model.eval(), inplace=True)
print(qat_model)

SimpleModel(
  (fc): QuantizedLinear(in_features=4, out_features=2, scale=0.010084597393870354, zero_point=141, qscheme=torch.per_tensor_affine)
)


### 6. 量化为何能减少存储/内存占用和加速推理速度？

- 低位宽（如int8）权重和激活比float32占用更少的存储空间。
- 低精度运算在硬件上通常更快。

In [9]:
import numpy as np
import time

# 1. 存储空间对比
float32_weights = np.random.randn(1000, 1000).astype(np.float32)
int8_weights = (float32_weights / float32_weights.max() * 127).astype(np.int8)

print(f"float32权重占用: {float32_weights.nbytes/1024/1024:.2f} MB")
print(f"int8权重占用: {int8_weights.nbytes/1024/1024:.2f} MB")

# 2. 推理速度对比（矩阵乘法）
x = np.random.randn(1000, 1000).astype(np.float32)

start = time.time()
_ = np.dot(x, float32_weights)
print(f"float32矩阵乘法耗时: {time.time()-start:.4f} 秒")

x_int8 = (x / x.max() * 127).astype(np.int8)
start = time.time()
_ = np.dot(x_int8, int8_weights)
print(f"int8矩阵乘法耗时: {time.time()-start:.4f} 秒")

float32权重占用: 3.81 MB
int8权重占用: 0.95 MB
float32矩阵乘法耗时: 0.0045 秒
int8矩阵乘法耗时: 0.3212 秒
int8矩阵乘法耗时: 0.3212 秒


### 7. PyTorch在Apple芯片上的float32与int8推理速度对比

下面代码对比同一个简单全连接模型在float32和int8（量化后）下的推理速度。

In [11]:
import torch
import torch.nn as nn
import torch.ao.quantization as quantization
import time

torch.backends.quantized.engine = 'qnnpack'  # Apple Silicon推荐

# 定义带量化stub的模型
class QuantModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.quant = quantization.QuantStub()
        self.fc = nn.Linear(1024, 1024)
        self.dequant = quantization.DeQuantStub()
    def forward(self, x):
        x = self.quant(x)
        x = self.fc(x)
        x = self.dequant(x)
        return x

# 生成测试数据
x = torch.randn(1000, 1024)

# 浮点模型推理
float_model = QuantModel().eval()
with torch.no_grad():
    start = time.time()
    for _ in range(10):
        _ = float_model(x)
    print(f"PyTorch float32模型推理耗时: {time.time()-start:.4f} 秒")

# 量化模型推理
quant_model = QuantModel().eval()
quant_model.qconfig = quantization.get_default_qconfig('qnnpack')
quantization.prepare(quant_model, inplace=True)
quant_model(x)  # 收集量化参数
quantization.convert(quant_model, inplace=True)
with torch.no_grad():
    start = time.time()
    for _ in range(10):
        _ = quant_model(x)
    print(f"PyTorch int8量化模型推理耗时: {time.time()-start:.4f} 秒")

PyTorch float32模型推理耗时: 0.0251 秒
PyTorch int8量化模型推理耗时: 0.0709 秒


**注意：为什么在Mac（Apple Silicon）上int8量化推理反而更慢？**

- PyTorch的int8量化推理在CPU上的加速效果，依赖于底层对int8运算的高度优化（如x86 AVX512 VNNI、ARM Neon/SDOT等指令集）。
- Apple Silicon（M1/M2/M3）目前PyTorch官方的int8量化后端（qnnpack）对int8没有做专门的硬件加速优化，float32反而能用到高效的BLAS库（如Accelerate）。
- 量化模型在推理时还涉及量化/反量化的额外操作，若底层没有专门优化，反而会拖慢速度。

**结论：**
- 量化的“推理加速”优势主要体现在移动端（如安卓ARM）、云端服务器（如Intel AVX512）、以及专用推理芯片（如NPU、TPU）上。
- 在Mac上用PyTorch体验不到int8的速度优势，但模型体积和内存占用依然会显著下降。
- 若要体验int8加速，可在x86服务器或使用ONNX Runtime、TensorRT等推理引擎。

#### x86平台（如Intel/AMD服务器）PyTorch量化推理加速示例

如果你在x86平台（如Intel Xeon/酷睿）上，PyTorch推荐使用'fbgemm'后端，通常能体验到int8推理加速。

In [ ]:
import torch
import torch.nn as nn
import torch.ao.quantization as quantization
import time

# 仅在x86平台推荐使用'fbgemm'后端
if torch.backends.quantized.engine != 'fbgemm':
    torch.backends.quantized.engine = 'fbgemm'

class QuantModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.quant = quantization.QuantStub()
        self.fc = nn.Linear(1024, 120000)
        self.dequant = quantization.DeQuantStub()
    def forward(self, x):
        x = self.quant(x)
        x = self.fc(x)
        x = self.dequant(x)
        return x

x = torch.randn(1000, 1024)

# 浮点模型推理
float_model = QuantModel().eval()
with torch.no_grad():
    start = time.time()
    for _ in range(10):
        _ = float_model(x)
    print(f"[x86] PyTorch float32模型推理耗时: {time.time()-start:.4f} 秒")

# 量化模型推理
quant_model = QuantModel().eval()
quant_model.qconfig = quantization.get_default_qconfig('fbgemm')
quantization.prepare(quant_model, inplace=True)
quant_model(x)
quantization.convert(quant_model, inplace=True)
with torch.no_grad():
    start = time.time()
    for _ in range(10):
        _ = quant_model(x)
    print(f"[x86] PyTorch int8量化模型推理耗时: {time.time()-start:.4f} 秒")

[x86] PyTorch float32模型推理耗时: 35.6941 秒  
[x86] PyTorch int8量化模型推理耗时: 26.5078 秒

## 8. 不同量化位宽的存储占用对比

常见量化位宽有 float32、float16、int8、int4、int3 等。位宽越低，模型存储和内存占用越小。

| 数据类型   | 每元素占用(bit) | 每元素占用(Byte) | 压缩率（相对float32） |
|------------|----------------|------------------|----------------------|
| float32    | 32             | 4                | 1x                   |
| float16    | 16             | 2                | 2x                   |
| int8       | 8              | 1                | 4x                   |
| int4       | 4              | 0.5              | 8x                   |
| int3       | 3              | 0.375            | 10.7x                |
| int2       | 2              | 0.25             | 16x                  |

- 例如：一个 1GB 的 float32 模型，int8 量化后约为 256MB，int4 量化后约为 128MB。
- 实际存储还会有量化参数、对齐等开销，但整体压缩比非常显著。

### 代码：不同位宽的权重占用

In [4]:
import torch
import torch.nn as nn
import torch.ao.quantization as quantization
import numpy as np
torch.backends.quantized.engine = 'qnnpack' 

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self, num_bits=8):
        super().__init__()
        self.fc = nn.Linear(4, 2) # 定义一个4输入2输出的全连接层
        self.num_bits = num_bits
    def forward(self, x):
        return self.fc(x)

# 不同位宽的模型
models = {bit: SimpleModel(bit).eval() for bit in [32, 16, 8, 4]}
# 随机输入
x = torch.randn(10, 4)

# 存储对比
print(f"{'数据类型':<10}{'每元素占用(Byte)':<20}{'模型占用内存（仅权重）':<30}")
for bit, model in models.items():
    # 权重量化
    qconfig = quantization.get_default_qconfig('qnnpack')
    model.qconfig = qconfig
    quantization.prepare(model, inplace=True)
    model(x)  # 前向传播收集参数
    quantization.convert(model, inplace=True)
    
    # 计算占用内存
    weight_memory = model.fc.weight().numel() * bit / 8
    print(f"{bit:<10}{bit/8:<20}{weight_memory:<30}")

# 示例：查看int8权重
model_int8 = models[8]
print("\nint8权重示例:", model_int8.fc.weight())

数据类型      每元素占用(Byte)         模型占用内存（仅权重）                   
32        4.0                 32.0                          
16        2.0                 16.0                          
8         1.0                 8.0                           
4         0.5                 4.0                           

int8权重示例: tensor([[-0.3581, -0.2374, -0.2452, -0.2725],
        [ 0.1713, -0.3386,  0.4943,  0.0000]], size=(2, 4), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.0038924638647586107,
       zero_point=0)


In [3]:
import numpy as np

# 假设有一个 1000x1000 的权重矩阵
shape = (1000, 1000)
float32_weights = np.random.randn(*shape).astype(np.float32)
float16_weights = float32_weights.astype(np.float16)
int8_weights = (float32_weights / np.abs(float32_weights).max() * 127).astype(np.int8)

# int4/int3 需要特殊打包，模拟存储空间
int4_bytes = float32_weights.size // 2  # 2个int4打包成1字节
int3_bytes = int(np.ceil(float32_weights.size * 3 / 8))  # 8个int3占3字节

print(f"float32占用: {float32_weights.nbytes/1024/1024:.2f} MB")
print(f"float16占用: {float16_weights.nbytes/1024/1024:.2f} MB")
print(f"int8占用: {int8_weights.nbytes/1024/1024:.2f} MB")
print(f"int4占用: {int4_bytes/1024/1024:.2f} MB (理论值)")
print(f"int3占用: {int3_bytes/1024/1024:.2f} MB (理论值)")

float32占用: 3.81 MB
float16占用: 1.91 MB
int8占用: 0.95 MB
int4占用: 0.48 MB (理论值)
int3占用: 0.36 MB (理论值)
